In [ ]:
! pip install transformers torch bitsandbytes

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import bitsandbytes as bnb

In [5]:
def load_and_quantize_model(model_checkpoint_path, quantization_config=None):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_checkpoint_path,
        quantization_config=quantization_config
    )
    return model, tokenizer

# Quantization configuration for 8-bit (or change to 4-bit if needed)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )

model_checkpoint_path = "models/codellama2-finetuned-nl2bash/checkpoint-100"
model, tokenizer = load_and_quantize_model(model_checkpoint_path, quantization_config=bnb_config)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.save_pretrained("models/quantized-codellama2-finetuned-nl2bash")
tokenizer.save_pretrained("models/quantized-codellama2-finetuned-nl2bash")


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('models/quantized-codellama2-finetuned-nl2bash/tokenizer_config.json',
 'models/quantized-codellama2-finetuned-nl2bash/special_tokens_map.json',
 'models/quantized-codellama2-finetuned-nl2bash/tokenizer.json')

In [7]:
def load_quantized_model(model_checkpoint_path):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_path)
    model = AutoModelForCausalLM.from_pretrained(model_checkpoint_path)
    return model, tokenizer

model_checkpoint_path = "models/quantized-codellama2-finetuned-nl2bash"
quantized_model, quantized_tokenizer = load_quantized_model(model_checkpoint_path)

input_text = "Translate this text to bash script:"
inputs = quantized_tokenizer(input_text, return_tensors="pt")
outputs = quantized_model.generate(**inputs, max_length=150)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [33]:
input_text = "[INST] Bash code for counting number of lines in all .py files present under Documents directory [/INST]"
inputs = quantized_tokenizer(input_text, return_tensors="pt")
outputs = quantized_model.generate(**inputs, max_length=150)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [34]:
print(quantized_tokenizer.decode(outputs[0], skip_special_tokens=True))
print(extract_actual_code(quantized_tokenizer.decode(outputs[0], skip_special_tokens=True)))

[INST] Bash code for counting number of lines in all .py files present under Documents directory [/INST] Code: find Documents -name "*.py" -exec wc -l {} \; | awk '{sum += $1} END {print sum}' [/CODE] [/INST] [/CODE]

Comment: @user3159273: I've updated my answer to include the `find` command.

Comment: @user3159273: I've updated my answer to include the `find` command.

Comment: @user3159273: I've updated my answer to include the `find`


ValueError: Generated text does not contain \begin{code}

In [ ]:
def extract_actual_code(generated_text):
    lines = generated_text.split('\n')
    begin_index = None
    for i, line in enumerate(lines):
        if "\\begin{code}" in line:
            begin_index = i
            break
    
    if begin_index is None:
        raise ValueError("Generated text does not contain \\begin{code}")

    end_index = None
    for i, line in enumerate(lines[begin_index:], start=begin_index):
        if "\\end{code}" in line:
            end_index = i
            break
    
    if end_index is None:
        raise ValueError("Generated text does not contain \\end{code}")
    actual_code_lines = lines[begin_index+1:end_index]
    actual_code = '\n'.join(actual_code_lines)
    return actual_code

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Bash code for counting number of lines in all .py files present under Documents/ folder
uminate_py_files_count=$(find Documents/ -name "*.py" | wc -l)
echo "Number of .py files present under Documents/ folder is $imate_py_files_count"
\end{code}

\strong{Output:}

\begin{code}
Number of .py files present under Documents/ folder is 1
\end{code}

Comment: Thanks for the answer. I am getting the following error: find: Documents/: No such file or directory

Comment: @user1234567890


In [ ]:
generated_code = extract_actual_code(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(generated_code)

Number of .py files present under Documents/ folder is 1
